## Importing Dependencies

In [15]:
from googleapiclient.discovery import build
import pandas as pd
import os
import webbrowser

## Creating Dataset

In [28]:
os.makedirs('data', exist_ok=True)
links_path = os.path.join('data', 'links.csv')
words_path = os.path.join('data', 'words.csv')

In [29]:
word_list = [""]

df = pd.DataFrame({'word': word_list})
df.to_csv(words_path, index=True, index_label='index')

In [17]:
def scrape_youtube_videos(query):
    api_key = os.environ.get("YOUTUBE_API_KEY")
    youtube = build('youtube', 'v3', developerKey=api_key)

    video_links = []
    video_ids = []
    next_page_token = None

    while True:
        search_response = youtube.search().list(
            q=query,
            type='video',
            part='id',
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in search_response['items']:
            video_links.append(
                'https://www.youtube.com/watch?v=' + item['id']['videoId'])
            video_ids.append(item['id']['videoId'])

        next_page_token = search_response.get('nextPageToken')

        if not next_page_token:
            break

    df = pd.DataFrame({'link': video_links, 'id': video_ids})
    df.to_csv(links_path, index=True, index_label='index')

In [18]:
scrape_youtube_videos("")

In [30]:
df = pd.read_csv(links_path)
df.head()

,index,link,id
0,0,https://www.youtube.com/watch?v=jmjthxWyJuE,jmjthxWyJuE
1,1,https://www.youtube.com/watch?v=0BHctoTXlDE,0BHctoTXlDE
2,2,https://www.youtube.com/watch?v=ELvYBPxLm3s,ELvYBPxLm3s
3,3,https://www.youtube.com/watch?v=D1eUb_iJj2I,D1eUb_iJj2I
4,4,https://www.youtube.com/watch?v=E9kMCCW8em4,E9kMCCW8em4


In [31]:
video_id = df['id']
video_id = video_id[:5]
video_id

0    jmjthxWyJuE
1    0BHctoTXlDE
2    ELvYBPxLm3s
3    D1eUb_iJj2I
4    E9kMCCW8em4
Name: id, dtype: object

## Playing Video in Browser

In [32]:
def play_youtube_video(video_id):
    for id in video_id:
        url = f"https://www.youtube.com/watch?v={id}"
        webbrowser.open(url)

In [33]:
play_youtube_video(video_id)